# Домашнее задание по рекомендательным системам

В данном домашнем задании вам предлагается реализовать User-based рекомендательную систему. Так же требуется реализовать несколько вспомогательных функций, шаблоны которых вы можете найти в `utils.py`.

Требования к выполнению задания:
- Реализация функции из `utils.py` засчитывается, только если пройдены все соответствующие тесты из `test.py`. Запуск тестов: <font color='red'>pytest test.py</font>. Для тестов вам потребуются библиотеки `numpy`, `scipy`, `pytest` и `hypothesis`.
- Плагиат запрещен. Если будет замечено, что часть задания списана, то 0 баллов ставится как списывающему, так и давшему списать.
- Если пользуетесь кодом из открытых источников, то указывайте ссылки, откуда взяли решение. Иначе такой код может быть воспринят как плагиат.
- При выполнении задания нельзя использовать библиотеку `scipy` и функцию `numpy.linalg.norm`

При запуске тестов могут появиться предупреждения: PearsonRConstantInputWarning и PearsonRNearConstantInputWarning. На них можно не обращать внимания.

Возможный максимум баллов за задание: 10 баллов <br>
Дедлайн: ??? <br>
Штраф: ??? - будет ли в курсе штраф? <br>
<br>
Для ускорения проверки, напишите здесь получившееся количество баллов: ...

## 1. Метрика сходства
<b>1.1. Реализация метрик (2 балла)</b>

Первое, с чем необходимо разобраться, при реализации User-based подхода, это с метрикой, с помощью которой будет решаться, насколько похожи пользователи. Вам предлагается реализовать 2 метрики: на основе евклидовой метрики и коэффициент корреляции Пирсона. Шаблоны для обоих функций можете найти в `utils.py`. Не забудьте проверить реализацию на тестах.

Евклидова метрика:
\begin{equation}
d(p,q)=\sqrt{(p_1-q_1)^2+(p_2-q_2)^2+\dots+(p_n-q_n)^2} = \sqrt{\sum_{k=1}^n (p_k-q_k)^2}
\end{equation}

В этом случае $d(p, q) \in [0, \infty)$, при этом если $d(p, q) \to 0$, то $sim(p, q) \to 1$. С учетом этого конечная формула будет выглядеть следующим образом:
\begin{equation}
sim(p, q) = \frac{1}{1 + d(p, q)}
\end{equation}
Так же в этой формуле не будет проблем с делением на 0.

Коэффициент корреляции Пирсона:
\begin{equation}
r_{xy} = \frac {\sum_{i=1}^{m} \left( x_i-\bar{x} \right)\left( y_i-\bar{y} \right)}{\sqrt{\sum_{i=1}^{m} \left( x_i-\bar{x} \right)^2 \sum_{i=1}^{m} \left( y_i-\bar{y} \right)^2}}
\end{equation}

<b>1.2. (1 балл)</b>

Рассмотрим пользователей $u$ и $v$. Им соотвествуют векторы $x_u$ и $x_v$, где $x_u[i] = r_{ui}$ и $x_v[i] = r_{vi}$. Из лекции известно, что похожесть между векторами $x_u$ и $x_v$ вычисляются только для тех индексов i, для которых существует и $r_{ui}$, и $r_{vi}$. То есть верно следуюющее:
\begin{equation}
sim(u, v) = sim(x_uI_{uv}, x_vI_{uv}),
\end{equation}
где $I_{uv} = [i | \exists r_{ui} \& \exists r_{vi}]$. При этом если $I_{uv} = \emptyset$, то $sim(u, v) \to -\infty$.

Реализуйте два новых метода, которые переиспользуют написанные вами `euclidean_distance` и `pearson_distance`, добавляющие условия на $x_u$ и $x_v$. Считается, что $x_u[i] = 0$, если $\nexists r_{ui}$. То же верно для $x_v$.

При реализации заданий можно как написать новые функции, так и использовать декораторы.

In [9]:
import numpy as np
from utils import euclidean_similarity, pearson_similarity
import pandas as pd

In [2]:
# your code (ﾉ>ω<)ﾉ :｡･:*:･ﾟ’★,｡･:*:･ﾟ’☆
def miss(x):
    x[x == -1] = 0
    return x


In [5]:
x = np.array([[1,2,3,8,-1],[7,4,-1,5,0]])
y = np.array([[7,4,-1,5,0],[7,4,-1,5,0]])

euclidean_similarity(miss(x), miss(x))

1.0

## 2. User-based method
<b>2.1. (3 балла)</b> 

Реализовать User-based подход, реализовав методы класса `UserBasedRecommendation`, основанного на использовании `NearestNeighbors`. В качестве метрики может для нахождения похожих пользователей может быть использована как евклидова метрика, так и коэффициент корреляции Пирсона.

Не забывайте, что `NearestNeighbors` ищет минимум расстояния между элементами, поэтому логично в качестве метрики при инициализации `NearestNeighbors` использовать обратную метрике схожести. То есть такую, что когда $sim(u, v) \to 1$, то $d(u, v) \to 0$. Например: $d(u, v) = 1 - sim(u, v)$

In [7]:
from sklearn.neighbors import NearestNeighbors
from typing import Optional

from collections import Counter


class UserBasedRecommendation:
    def __init__(self, metric: str = 'euclidean', n_recommendations: int = 5, alpha: float = 0.8):
        """
        Args:
            metric: name of metric: ['euclidean', 'pearson']
            n_recommendations: number of recommendations. Also can be specified self.make_recommendation
            alpha: similarity threshold: if sim(u, v) > alpha then u and v are similar
        """
        self.metric = metric
        self.n_recommendations = n_recommendations
        self.alpha = alpha

    def fit(self, X: np.array):
        """
        Args:
            X: matrix N x M where X[u, i] = r_{ui} if r_{ui} exists else X[u, i] = 0
        """
        self.X = X

    def find_closest_users(self, user_id: int, n_closest_users: int):
        # your code (ﾉ>ω<)ﾉ :｡･:*:･ﾟ’★,｡･:*:･ﾟ’☆
        scores = []
        if self.metric == 'euclidean':
            for i in range(len(self.X)):
                scores.append((1 - euclidean_similarity(miss(self.X[user_id]), miss(self.X[i])), i))
            scores = (sorted(scores))[1:n_closest_users+1]
        elif self.metric == 'pearson':
            for i in range(len(self.X)):
                scores.append((pearson_similarity(miss(self.X[user_id]), miss(self.X[i])), i))
            scores = (sorted(scores, reverse = True))[1:n_closest_users+1]
        
        scores = [x for x in scores if x[0] > self.alpha]
        ind = np.array(list(map(lambda x: x[1], scores)))
        self.ind = ind
        return scores
    def make_recommendation(self, user_id: int, n_recommendations: Optional[int] = None):
        """
        Args:
            user_id: user id to whom you want to recommend
            n_recommendations: number of recommendations
        """
        # your code (ﾉ>ω<)ﾉ :｡･:*:･ﾟ’★,｡･:*:･ﾟ’☆
        if self.metric == 'euclidean':
            rating = []
            for i in range(self.X.shape[1]):
                sim = 0
                sum_sim = 0
                for j in self.ind:
                    sum_sim += np.abs(euclidean_similarity(miss(self.X[j]), miss(self.X[user_id])))
                    sim += euclidean_similarity(miss(self.X[j]), miss(self.X[user_id]))*(self.X[j,i] - np.mean(self.X[j]))
                if self.X[user_id, i] == 0:
                    rating.append((np.mean(self.X[user_id][self.X[user_id] > 0]) + sim / sum_sim, i))
        elif self.metric == 'pearson':
            rating = []
            for i in range(self.X.shape[1]):
                sim = 0
                sum_sim = 0
                for j in self.ind:
                    sum_sim += np.abs(pearson_similarity(miss(self.X[j]), miss(self.X[user_id])))
                    sim += pearson_similarity(miss(self.X[j]), miss(self.X[user_id]))*(self.X[j,i] - np.mean(self.X[j]))
                if self.X[user_id, i] == 0:
                    rating.append((np.mean(self.X[user_id][self.X[user_id] > 0]) + sim / sum_sim, i))
        return sorted(rating, reverse = True)[:n_recommendations]

<b>2.2. (1 балла)</b>

Приведите пример, для которого использование разных метрик будет давать разные рекомендации. Объясните свой пример.

In [10]:
X = np.array([[-1, 3, 2, -1, -1, 6], [-1, 4, 2, 7, -1, 6], [5, 1, 3, -1, 7, -1], [5, 4, -1, 4, 9, 7], [7, -1, 7, 4, 6, 5]]) 

pd.DataFrame(X, index = ['USER0', 'USER1','USER2', 'USER3', 'USER4'], columns = ['M0', 'M1', 'M2', 'M3','M4', 'M5'])

,M0,M1,M2,M3,M4,M5
USER0,-1,3,2,-1,-1,6
USER1,-1,4,2,7,-1,6
USER2,5,1,3,-1,7,-1
USER3,5,4,-1,4,9,7
USER4,7,-1,7,4,6,5


#### Euclidean

In [11]:
rec = UserBasedRecommendation(metric='euclidean', alpha = -1000)
rec.fit(X)
rec.find_closest_users(0,2)

[(0.8761006569007046, 1), (0.914703462326635, 2)]

В данном примере мы видим, что метрика 'euclidean' показывает максимальную схожесть USER0 и USER2, хотя мы видим что на самом деле для 0го юзера самым близким будем USER1, ведь у них оценки для первого, второго и пятого фильма практически идентичны, в то время как с USER2 и них явные различия во вкусах(на примере оценок первого фильма).

In [12]:
rec.make_recommendation(0,3)

[(4.849720784036919, 3), (3.5580144799574005, 4), (2.74254395196876, 0)]

Получаем какие-то такие рекомендации

#### Pearson

In [13]:
rec = UserBasedRecommendation(metric='pearson', alpha = -1000)
rec.fit(X)
rec.find_closest_users(0,2)

[(0.4774611427462813, 1), (0.02267742779480786, 3)]

Метрика 'pearson' работает гораздо лучше. Во-первых, алгоритм 'понимает', что 1й пользователь самый близкий к 0му, а также вторым по близости определяет 3го пользователя, и это действително похоже на правду, если посмотреть на их оценки фильмов, то кажется что вкусы у них +- схожи

In [14]:
rec.make_recommendation(0,3)

[(7.288402649567145, 3), (0.8325101221087725, 4), (0.6511409645948967, 0)]

Полученная рекомендация для 3го фильма выглядит действительно хорошо, посмотрев на ближайших по схожести пользователей видно, что оценка 7 для этого фильма правдоподобна. С остальными двумя фильмами ситуация хуже конечно, но нас интересует порядок, а с ним все в порядке.

## 3. Оценка качества
<b>3.1. (1 балл)</b>

Реализуйте Average Precision at k и Mean Average Precision at k. Шаблоны можете найти в `utils.py`.
\begin{align*}
AP@K = \frac{1}{m}\sum_{k=1}^K P(k)*rel(k), \\
MAP@K = \frac{1}{|U|}\sum_{u=1}^{|U|}(AP@K)_u
\end{align*}
где $P(k)$ - Precision at k, $rel(k) = 1$, если рекомендация релевантна, иначе $rel(k) = 0$.

## 4. Применение модели
<b>4.1. (2 балла)</b>

Выгрузите датасет `ratings_small.csv`: https://www.kaggle.com/rounakbanik/the-movies-dataset#ratings_small.csv

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
data = pd.read_csv('ratings_small.csv', index_col=False)
data.shape

(100004, 4)

In [18]:
data.userId.min(), data.userId.max(), len(data.userId.unique())

(1, 671, 671)

In [19]:
data.movieId.min(), data.movieId.max(), len(data.movieId.unique())

(1, 163949, 9066)

Для простоты работы с данными, измените нумерацию пользователей и фильмов так, чтобы нумерация начиналась с 0 и шла непрерывно.

In [20]:
movie_to_idx = {movie_name : idx for idx, movie_name in enumerate(data.movieId.unique())}
idx_to_movie = {i: movie_name for movie_name, i in movie_to_idx.items()}

In [21]:
user_to_idx = {user_id : idx for idx, user_id in enumerate(data.userId.unique())}
idx_to_user = {i: user for user, i in user_to_idx.items()}

In [22]:
data.movieId = [movie_to_idx[idx] for idx in data.movieId]
data.userId = [user_to_idx[idx] for idx in data.userId]

In [23]:
data.userId.min(), data.userId.max(), len(data.userId.unique())

(0, 670, 671)

In [24]:
data.movieId.min(), data.movieId.max(), len(data.movieId.unique())

(0, 9065, 9066)

Удалим для наиболее активных пользователей 5 оценок

In [25]:
active_users = data.userId.value_counts()[:10].index
test_data = pd.DataFrame([], columns=data.columns)
for user_id in active_users:
    _, test = train_test_split(data[data.userId == user_id], test_size=5, random_state=42)
    test_data = test_data.append(test, ignore_index=True)
    data = data[~((data.userId == user_id) & (data.movieId.isin(test.movieId.values)))]
data.shape, test_data.shape

((99954, 4), (50, 4))

Преобразуем данные в таблицу `X`, с которой может работать `UserBasedRecommendation`, где $X_{ui} = r_{ui}$, если пользователь $u$ поставил оценку фильму $i$, и $X_{ui} = 0$, если пользователь $u$ не проставил оценку фильму $i$.

Вам может пригодиться `csr_matrix`.

In [26]:
import scipy.sparse as scp

In [27]:
def get_sparse(data):
    return scp.coo_matrix(
        (
            data['rating'],  # оценки пользователей
            (data['userId'], data['movieId'])  # id пользователей и id аниме, для которых известны оценки
        ), 
        shape=(len(user_to_idx), len(movie_to_idx))  # размеры матрицы рейтингов
    ).tocsr()

In [28]:
len(user_to_idx)

671

In [29]:
train_sp = get_sparse(data)
train_sp

<671x9066 sparse matrix of type '<class 'numpy.float64'>'
	with 99954 stored elements in Compressed Sparse Row format>

In [30]:
train_dense = train_sp.todense().A
train_dense

array([[2.5, 3. , 3. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       ...,
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ]])

Для пользователей, у которых были удалены фильмы, найдите топ 100 фильмов, который должен посмотреть каждый из этих пользователей, используя `UserBasedRecommendation`. Не забудьте подобрать параметр alpha.

In [31]:
from tqdm import tqdm
user_ids = np.unique(test_data.userId)

preds = []
rec = UserBasedRecommendation(alpha = 0.1)
rec.fit(train_dense)
for i in tqdm(user_ids):
    rec.find_closest_users(i,20)
    rec_mov = rec.make_recommendation(i,100)
    preds.append(list(map(lambda x: x[1], rec_mov)))

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [04:50<00:00, 29.09s/it]


Используя метрику `MAP@5`, `MAP@10` и `MAP@100`, определите, насколько эффективна user-based рекомендательная система для данной задачи.

In [32]:
from utils import mapk, apk

In [33]:
test_actual = []
for i in range(len(user_ids)):
    test_actual.append(np.array(test_data.movieId[test_data.userId == user_ids[i]]))

In [34]:
apk(np.array(test_actual[0]), np.array(preds[0]), 100)

0.2

In [35]:
for i in [5, 10, 100]:
    print(f'MAP@{i}: {mapk(test_actual, preds, i)}')

MAP@5: 0.08
MAP@10: 0.08
MAP@100: 0.09284802330306383


Как можно улучшить работу модели?

<b>Ответ:</b>...

Переберем гиперпараметры

In [98]:
results = []
for metric in ['pearson', 'euclidean']:
    for alpha in tqdm([0.2, 0.1, 0.01]):
        preds = []
        rec = UserBasedRecommendation(alpha = alpha, metric = metric)
        rec.fit(train_dense)
        for i in user_ids:
            rec.find_closest_users(i,20)
            rec_mov = rec.make_recommendation(i,100)
            preds.append(list(map(lambda x: x[1], rec_mov)))
        res = []
        for i in [5, 10, 100]:
            res.append(mapk(test_actual, preds, i))
        results.append(res)

100%|███████████████████████████████████████████████████████████████████████████████████| 3/3 [09:12<00:00, 184.17s/it]


In [99]:
results

[[0.09666666666666666, 0.09666666666666666, 0.10663872121618598],
 [0.09666666666666666, 0.09666666666666666, 0.10663872121618598],
 [0.09666666666666666, 0.09666666666666666, 0.10663872121618598],
 [0.08, 0.08, 0.09284802330306383],
 [0.08, 0.08, 0.09284802330306383],
 [0.08, 0.08, 0.09284802330306383]]

Комбинация 'pearson' и alpha = 0.2/0.1/0.01 дают лучший результат

In [100]:
preds = []
rec = UserBasedRecommendation(metric = 'pearson', alpha = 0.1)
rec.fit(train_dense)
for i in tqdm(user_ids):
    rec.find_closest_users(i,20)
    rec_mov = rec.make_recommendation(i,100)
    preds.append(list(map(lambda x: x[1], rec_mov)))

for i in [5, 10, 100]:
    print(f'MAP@{i}: {mapk(test_actual, preds, i)}')

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [14:13<00:00, 85.32s/it]

MAP@5: 0.09666666666666666
MAP@10: 0.09666666666666666
MAP@100: 0.10663872121618598
